# Flower Recognition Project
This notebook trains a ResNet-18 model to classify 5 types of flowers using a Kaggle dataset.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Cell 1: Install required libraries
# Install PyTorch, torchvision, and Gradio quietly to avoid cluttering output
!pip install torch torchvision gradio -q
print('Libraries installed successfully!')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 70.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.7 MB/s eta 0:00:00
Libraries installed successfully!


In [ ]:
# Cell 2: Mount Google Drive and import libraries
from google.colab import drive
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
import torchvision
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import gradio as gr
from torch.optim.lr_scheduler import ReduceLROnPlateau


# Define dataset path (UPDATE THIS to your dataset path in Google Drive)
dataset_path = '/content/drive/MyDrive/Flowers/flowers'

print('Drive mounted and libraries imported!')

Drive mounted and libraries imported!


In [ ]:
# Cell 3: Define transformations and load dataset
# Transformations for training (with augmentation to prevent overfitting)
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to 224x224 for ResNet
    transforms.RandomHorizontalFlip(),  # Augmentation: random flip
    transforms.RandomRotation(15),  # Augmentation: random rotation
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),  # Augmentation: color jitter
    transforms.ToTensor(),  # Convert image to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet normalization
])

# Transformations for validation (no augmentation)
val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load dataset using ImageFolder
dataset = datasets.ImageFolder(root=dataset_path, transform=train_transforms)

# Split dataset into training (80%) and validation (20%)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Apply validation transforms to validation dataset
val_dataset.dataset.transform = val_transforms

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)

# Load class names
class_names = dataset.classes
num_classes = len(class_names)
print(f'Classes: {class_names}')
print(f'Number of classes: {num_classes}')
print(f'Training samples: {train_size}, Validation samples: {val_size}')

Classes: ['daisy', 'dandelion', 'rose', 'sunflower', 'tulip']
Number of classes: 5
Training samples: 2060, Validation samples: 515


In [ ]:
# Cell 4: Set up ResNet-18 model with dropout
# Check for GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# Define custom ResNet-18 with dropout to prevent overfitting
class ResNet18WithDropout(nn.Module):
    def __init__(self, num_classes):
        super(ResNet18WithDropout, self).__init__()
        self.resnet = torchvision.models.resnet18(pretrained=True)
        in_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Sequential(
            nn.Dropout(0.5),  # Dropout to prevent overfitting
            nn.Linear(in_features, num_classes)
        )

    def forward(self, x):
        return self.resnet(x)

# Initialize model, loss function, optimizer, and scheduler
model = ResNet18WithDropout(num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)

print('Model, loss function, optimizer, and scheduler initialized!')

Using device: cuda


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 76.3MB/s]


Model, loss function, optimizer, and scheduler initialized!


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [7]:
# Cell 5: Train the model with early stopping
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=5):
    best_val_loss = float('inf')
    patience = 5  # Early stopping patience
    patience_counter = 0

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()

        train_loss = train_loss / train_total
        train_acc = 100 * train_correct / train_total

        # Validation phase
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)

                val_loss += loss.item() * images.size(0)
                _, predicted = torch.max(outputs, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

        val_loss = val_loss / val_total
        val_acc = 100 * val_correct / val_total

        # Step the scheduler
        scheduler.step(val_loss)

        # Print epoch results
        print(f'Epoch {epoch+1}/{num_epochs}:')
        print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%')
        print(f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%')

        # Early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            torch.save(model.state_dict(), '/content/drive/MyDrive/best_flower_model.pth')
            print('Best model saved!')
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print('Early stopping triggered!')
                break

    return model

# Train the model
model = train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=5)
print('Training completed!')

Epoch 1/20:
Train Loss: 0.8599, Train Acc: 69.66%
Val Loss: 1.4087, Val Acc: 67.38%
Best model saved!
Epoch 2/20:
Train Loss: 0.5409, Train Acc: 80.53%
Val Loss: 0.8825, Val Acc: 71.84%
Best model saved!
Epoch 3/20:
Train Loss: 0.3922, Train Acc: 86.17%
Val Loss: 0.5621, Val Acc: 83.50%
Best model saved!
Epoch 4/20:
Train Loss: 0.4273, Train Acc: 85.24%
Val Loss: 0.9440, Val Acc: 73.59%
Epoch 5/20:
Train Loss: 0.2971, Train Acc: 89.61%
Val Loss: 0.4377, Val Acc: 84.08%
Best model saved!
Epoch 6/20:
Train Loss: 0.2132, Train Acc: 91.84%
Val Loss: 0.6124, Val Acc: 78.45%
Epoch 7/20:
Train Loss: 0.1896, Train Acc: 93.50%
Val Loss: 0.5794, Val Acc: 82.91%
Epoch 8/20:
Train Loss: 0.2229, Train Acc: 92.38%
Val Loss: 0.6277, Val Acc: 80.00%
Epoch 9/20:
Train Loss: 0.1279, Train Acc: 95.29%
Val Loss: 0.8748, Val Acc: 76.70%
Epoch 10/20:
Train Loss: 0.0587, Train Acc: 98.11%
Val Loss: 0.3768, Val Acc: 88.16%
Best model saved!
Epoch 11/20:
Train Loss: 0.0286, Train Acc: 99.51%
Val Loss: 0.3612, 

In [8]:
# Cell 6: Load the best model
model.load_state_dict(torch.load('/content/drive/MyDrive/best_flower_model.pth'))
model.eval()
print('Best model loaded!')

Best model loaded!


In [9]:
# Cell 7: Define prediction function
def predict_image(image_path, model, class_names, transform):
    model.eval()
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        outputs = model(image)
        probabilities = torch.softmax(outputs, dim=1)[0]
        _, predicted = torch.max(outputs, 1)

    predicted_class = class_names[predicted.item()]
    confidence = probabilities[predicted.item()].item()
    prob_dict = {class_names[i]: prob.item() for i, prob in enumerate(probabilities)}
    return predicted_class, confidence, prob_dict

print('Prediction function defined!')

Prediction function defined!


In [11]:
# Cell 8: Set up and launch Gradio interface
def gradio_predict(image):
    transform = val_transforms
    predicted_class, confidence, prob_dict = predict_image(image, model, class_names, transform)
    result = f'Predicted Flower: {predicted_class}\nConfidence: {confidence:.2%}\n\nProbabilities:\n'
    for cls, prob in prob_dict.items():
        result += f'{cls}: {prob:.2%}\n'
    return result

iface = gr.Interface(
    fn=gradio_predict,
    inputs=gr.Image(type='filepath'),
    outputs='text',
    title='Flower Recognition',
    description='Upload an image of a flower to classify it as one of: daisy, dandelion, rose, sunflower, or tulip.'
)
iface.launch(share=False)
print('Gradio interface launched!')

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

Gradio interface launched!
